In [1]:
import chess
import chess.uci
import chess.pgn
import sys,os
import numpy as np
import torch
import io
import pandas as pd
import concurrent
import dill
%load_ext autoreload

%autoreload 2

In [2]:
import torch,dill
import matplotlib.pyplot as plt
from oil.utils.utils import Eval, cosLr
from chess_dataset import ChessDataset

In [3]:
from gameTrainer2D import baseGameTrainTrial,makeSimpleTrainer

In [4]:
with open(os.path.expanduser('~/games/chess/data/{}_train_0.pkl'.format('chess_3000k_0.2s')), 'rb') as file:
    pgns = dill.load(file)

In [5]:
D = ChessDataset(os.path.expanduser('~/games/chess/data/{}_train_0.pkl'.format('chess_3000k_0.2s')))

In [ ]:
Trial = baseGameTrainTrial(strict=True)

In [ ]:
in_cfg ={
    'num_epochs':30,'dataset':'chess_3000k_0.2s',
    'bs': 128,
    'trainer_config':{'value_weight':.01,'log_args':{'minPeriod':0.1,'no_print':True}},
    'opt_constr':lambda params: torch.optim.Adam(params,lr=1e-3),#lambda params: torch.optim.SGD(params,**{'lr':.03, 'momentum':.9, 'weight_decay':1e-4}),#
    'net_config':{'coords':True,'num_blocks':20,'k':128},
    'lr_sched':cosLr(30),
}

In [ ]:
#cfg,outcome=Trial(in_cfg)
T = makeSimpleTrainer(in_cfg)
for i in range(10):
    T.train(2)
    saved_at = T.logger.save_object(T,suffix='checkpoints/c{}.trainer'.format(T.epoch+1))

In [ ]:
saved_at = T.logger.save_object(T,suffix='checkpoints/c{}.trainer'.format(T.epoch+1))

In [ ]:
#T = torch.load(cfg['saved_at'],pickle_module=dill)
T.logger.scalar_frame

In [ ]:
plt.style.use('ggplot')
plt.rcParams.update({'font.size': 14})
f = plt.figure()
T.logger.scalar_frame.plot()

In [ ]:
from chess_dataset import ChessDataset
import os
import torch
from torch.utils.data import DataLoader
from oil.utils.utils import to_device_layer,LoaderTo,Eval
import numpy as np

In [ ]:
D = ChessDataset(os.path.expanduser('~/games/chess/chess_train.pkl'))

In [ ]:
vs = np.array([D[i][2] for i in range(300)])

In [ ]:
plt.hist(vs)

In [ ]:
vs.mean()

In [ ]:
loader = LoaderTo(DataLoader(D,batch_size=5,shuffle=False),torch.device('cuda'))

In [ ]:
with Eval(T.model), torch.no_grad():
    values = np.concatenate([T.model(mb[0])[0].cpu().data.numpy() for mb in loader])

In [ ]:
values.mean()

In [ ]:
plt.hist(values)

In [ ]:
vs2 = np.array()

In [ ]:
vs2

In [ ]:
mb[2].type()